# Model Trainer Notebook

Modifying file path.

In [3]:
import os

In [4]:
%pwd

'c:\\Users\\RaviB\\GitHub\\SleepEfficiencyML\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\RaviB\\GitHub\\SleepEfficiencyML'

## Model Tuning

In [12]:
import pandas as pd
from sklearn.linear_model import ElasticNet

In [8]:
train_data = pd.read_csv("artifacts/data_transformation/train.csv")
test_data = pd.read_csv("artifacts/data_transformation/test.csv")

train_x = train_data.drop(['Sleep efficiency'], axis=1)
test_x = test_data.drop(['Sleep efficiency'], axis=1)
train_y = train_data[['Sleep efficiency']]
test_y = test_data[['Sleep efficiency']]

In [13]:
lr = ElasticNet(alpha=0.4, l1_ratio=0.8, random_state=42)
lr.fit(train_x, train_y)

ElasticNet(alpha=0.4, l1_ratio=0.8, random_state=42)

In [17]:
data = test_x.iloc[5]
data

Age                     30.0
Gender                   0.0
Bedtime                  2.0
Wakeup time             11.0
Sleep duration           9.0
Awakenings               0.0
Caffeine consumption     1.0
Alcohol consumption      0.0
Smoking status           0.0
Exercise frequency       0.0
Name: 5, dtype: float64

In [18]:
feature_names = list(test_x.columns)
feature_names

['Age',
 'Gender',
 'Bedtime',
 'Wakeup time',
 'Sleep duration',
 'Awakenings',
 'Caffeine consumption',
 'Alcohol consumption',
 'Smoking status',
 'Exercise frequency']

In [21]:
data_df = pd.DataFrame([data], columns=feature_names)

In [22]:
lr.predict(data_df)

array([0.78707965])

In [24]:
for i in range(0, 15):
    data = test_x.iloc[i]
    data_df = pd.DataFrame([data], columns=feature_names)
    print(lr.predict(data_df))

[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]
[0.78707965]


## Modularizing Model Training

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [21]:
from sleep_efficiency.constants import *
from sleep_efficiency.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        
        return model_trainer_config

In [28]:
import pandas as pd
import os
from sleep_efficiency import logger
from sklearn.linear_model import ElasticNet
import joblib

In [29]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [31]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-26 10:30:16,444: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-26 10:30:16,446: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-26 10:30:16,450: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-26 10:30:16,452: INFO: common: created directory at artifacts]
[2024-06-26 10:30:16,453: INFO: common: created directory at artifacts/model_trainer]
